In [16]:
import pandas as pd
import sqlalchemy as sqlalc
import requests
import time 
import tqdm

Extract Data

In [17]:
df = pd.read_xml('https://discogs-data-dumps.s3-us-west-2.amazonaws.com/data/2023/discogs_20230101_labels.xml.gz')
df

,images,id,name,contactinfo,profile,data_quality,urls,sublabels,parentLabel
0,NaN,1,Planet E,Planet E Communications\r\nP.O. Box 27218\r\nD...,[a=Carl Craig]'s classic techno label founded ...,Needs Vote,NaN,NaN,None
1,NaN,2,Earthtones Recordings,Seasons Recordings\r\n2236 Pacific Avenue\r\nS...,California deep house label founded by [a=Jami...,Correct,NaN,NaN,None
2,NaN,3,Seasons Recordings,"Seasons Recordings\r\nCosta Mesa, CA 92627\r\n...",California deep-house label founded by [a=Jami...,Needs Vote,NaN,NaN,None
3,NaN,4,Siesta Music,Siesta Records\r\n1913 Via Encantadoras\r\nSan...,None,Needs Vote,NaN,NaN,None
4,NaN,5,Svek,Svek office \r\nStephan Grieder \r\nFax: +46 (...,None,Correct,NaN,NaN,Goldhead Music
...,...,...,...,...,...,...,...,...,...
1946241,NaN,3078049,5th Gear Productions,None,None,Needs Major Changes,NaN,NaN,None
1946242,NaN,3078052,"Tunnell, Jimi Music",None,None,Needs Major Changes,NaN,NaN,None
1946243,NaN,3078055,Warrior Gospel Records,None,None,Needs Major Changes,NaN,NaN,None
1946244,NaN,3078058,Allegretto Music,None,None,Needs Major Changes,NaN,NaN,None


Filter Data

In [18]:
df = df.loc[(df['data_quality'] == 'Complete and Correct') & (~df['contactinfo'].isna())]
df.head()

,images,id,name,contactinfo,profile,data_quality,urls,sublabels,parentLabel
725,NaN,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,NaN,NaN,Ministry Of Sound
2878,NaN,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,NaN,NaN,Global Beat Records
2935,NaN,3203,Tinnitus Tonträger,Tinnitus Tonträger\r\nMarc Göttling\r\nWiesent...,Style: hard kicking minimal techno traxx.,Complete and Correct,NaN,NaN,None
3020,NaN,3295,Taaach! Recordings,Taaach!\r\nEdition Chrome\r\nPostfach 10 50 54...,German Trance Label\r\nDistributed by InterGro...,Complete and Correct,NaN,NaN,None
3505,NaN,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,NaN,NaN,Massive Records


Drop Uneccessary Columns

In [19]:
df = df.drop(columns=['images', 'data_quality', 'urls', 'sublabels'])

In [20]:
df = df.set_index('id')

In [21]:
num_labels = df.shape[0]
print(num_labels)

430


In [ ]:
token = 'UNuvnlhbZSwMsuuoyRTuoBHyLGqACNqhOjxuVpVo'
other_columns = pd.DataFrame()
for label_id in tqdm.tqdm(df.index):
    successful = False
    while not successful:
        try:
            res = requests.get(f'https://api.discogs.com/labels/{label_id}/releases', params={'token': token}).json()
            releases = res['releases']
            successful = True
            time.sleep(1)  # 1 request per second
        except Exception:
            print(f'Request failed on label ID: {label_id}. Retry in 60 seconds...')
            time.sleep(60)
    n_releases = len(releases)
    years = [release['year'] for release in releases]
    min_year = min(years) if len(years) > 0 else None
    max_year = max(years) if len(years) > 0 else None
    next_row = pd.DataFrame({'num_of_releases': [n_releases], 'min_release_year': [min_year], 'max_release_year': [max_year]}, index=[label_id])
    other_columns = pd.concat([other_columns, next_row], axis='rows')

  0%|                                                   | 0/430 [00:00<?, ?it/s]

In [ ]:
df = pd.concat([df, other_columns], axis='columns')

In [ ]:
user = 'postgres'
password = 'hellosql'
host = 'localhost'
port = 5432
database_name = 'Record Labels'
connection_url = f'postgresql://{user}:{password}@{host}:{port}/{database_name}'
engine = sqlalc.create_engine(connection_url)
with engine.connect() as con:
    df.to_sql('labels', con, index=False, if_exists='replace')

In [ ]:
with engine.connect() as con:
    display(pd.read_sql('SELECT * FROM labels', con))